# Scraping Asylum Filings

https://trac.syr.edu/phptools/immigration/asyfile/

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import os

# webdriver_path =  "/Users/joaburkh/Downloads/geckodriver"
# webdriver_path = '/Users/joaburkh/Downloads/chromedriver-mac-x64/chromedriver'

ops = Options()
# ops.add_argument('--headless')
browser = webdriver.Firefox()
browser.get('https://trac.syr.edu/phptools/immigration/asyfile/')
browser.implicitly_wait(15)

In [4]:
# Get top-level data list buttons
axis_buttons = browser.find_elements(By.CLASS_NAME, 'truncate')
axis_button_text = [e.text for e in axis_buttons]
for b in axis_buttons:
    print(b.text)

Immigration Court State
Custody
Represented


In [6]:
# Open first axis option list and get list entries
axis_buttons[0].click()

# Set the first axis to 'Fiscal Year of Decision'
axis_value = 'Fiscal Year Application Filed'
axis1_options = [e for e in browser.find_elements(By.CLASS_NAME, "truncate") if e.text not in axis_button_text]
# axis1_options = browser.find_elements(By.ID, 'headlessui-listbox-options-2')
axis1_options_text = [e.text for e in axis1_options]
axis1_options[axis1_options_text.index(axis_value)].click()

# Re-calculate axis button text
axis_button_text = [e.text for e in axis_buttons]

In [7]:
# Open second axis option list and get list entries
axis_buttons[1].click()

# Set the second axis to 'Nationality'
axis_value = 'Nationality'
axis2_options = [e for e in browser.find_elements(By.CLASS_NAME, "truncate") if e.text not in axis_button_text]
axis2_options_text = [e.text for e in axis2_options]
axis2_options[axis1_options_text.index(axis_value)].click()

# Re-calculate axis button text
axis_button_text = [e.text for e in axis_buttons]

In [8]:
# Open third axis option list and get list entries
axis_buttons[2].click()

# Set the second axis to 'Nationality'
axis_value = 'Outcome'
axis2_options = [e for e in browser.find_elements(By.CLASS_NAME, "truncate") if e.text not in axis_button_text]
axis2_options_text = [e.text for e in axis2_options]
axis2_options[axis1_options_text.index(axis_value)].click()

# Re-calculate axis button text
axis_button_text = [e.text for e in axis_buttons]

In [9]:
# Get list of tables
tables = browser.find_elements(By.CLASS_NAME, 'table-fixed')

In [14]:
import time
# Get list of entries along first axis (to be clicked on one by one)
table1_rows = tables[0].find_elements(By.CLASS_NAME, 'flex-row')
table1_rows = [row for row in table1_rows if row.text != '' and row.text.find('All') == -1]

# Initialize data dict
data = {}

# Iterate over each year
for t1_row in table1_rows:
    # Calculate year and initialize data dictionary entry
    year = int(t1_row.text.rsplit(' ', 1)[0])
    data[year] = {}

    # Click on the row
    t1_row.click()

    # Recalculate Tables
    tables = browser.find_elements(By.CLASS_NAME, 'table-fixed')

    # Get list of entries along second axis (to be clicked on one by one)
    table2_rows = tables[1].find_elements(By.CLASS_NAME, 'flex-row')
    table2_rows = [row for row in table2_rows if row.text != '' and row.text.find('All') == -1]
    
    # Iterate over each nationality for current year
    for t2_row in table2_rows:
        # Calculate Nationality
        nationality = t2_row.text.rsplit(' ', 1)[0]

        # Click on the row
        t2_row.click()

        # Recalculate Tables
        tables = browser.find_elements(By.CLASS_NAME, 'table-fixed')

        # Get a list of entries along third axis (to be copied as-is)
        time.sleep(0.2)
        table3_rows = tables[2].find_elements(By.CLASS_NAME, 'flex-row')
        table3_rows = [row for row in table3_rows if row.text != '' and row.text.find('All') == -1]
        table3_text = [row.text.replace(',', '').rsplit(' ', 1) for row in table3_rows]
        table3_text = {row[0]: int(row[1]) for row in table3_text}

        # Add to data dictionary
        data[year][nationality] = table3_text

In [16]:
data.keys()

dict_keys([2022, 2019, 2020, 2018, 2017, 2021, 2016, 2002, 2015, 2003, 2001, 2023, 2004, 2007, 2014, 2006, 2012, 2005, 2013, 2011, 2008, 2009, 2010])

In [19]:
# Export to HDF
import pandas as pd 
from pathlib import Path

data_df = pd.concat({k: pd.DataFrame(v).T for k, v in data.items()}, axis=0) # https://stackoverflow.com/a/54300940
data_df.to_hdf(Path.cwd() / 'TRAC_AsylumFilings_Raw.hdf', key='AsylumFilings_2001to2023')

In [21]:
data_df = pd.read_hdf('TRAC_AsylumFilings_Raw.hdf', key='AsylumFilings_2001to2023')
# data_df.rename_axis(('Year', 'Nationality'), inplace=True)                        # rename indices for clarity

# Pre-Processing Data

In [1]:
import pandas as pd

# Import Data
data_df = pd.read_hdf('TRAC_AsylumFilings_Raw.hdf')

# Set index name
data_df.index.rename(names=['Year', 'Nationality'], level=[0, 1], inplace=True)
data_df

C:\Users\Joseph\AppData\Local\Temp\ipykernel_33676\1372840210.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Pending  Grant Relief  Removal Order  Other Closure  \
Year Nationality                                                             
2022 Venezuela         34852.0        1088.0          399.0          319.0   
     Honduras          24163.0         395.0         2084.0         1189.0   
     Cuba              25745.0         147.0          174.0          358.0   
     Guatemala         19222.0         434.0         1815.0         1729.0   
     Ecuador           13052.0         802.0         3147.0          624.0   
...                        ...           ...            ...            ...   
2010 Swaziland             NaN           NaN            NaN            NaN   
     Falkland Islands      NaN           NaN            1.0            NaN   
     Slovenia              NaN           NaN            1.0            NaN   
     Bahrain               NaN           NaN            NaN            NaN   
     French Guiana         NaN           NaN            1.0            NaN   

                       Terminate Proceedings  Voluntary Departure  \
Year Nationality                                                    
2022 Venezuela                          67.0                 23.0   
     Honduras                          243.0                 70.0   
     Cuba                              150.0                  NaN   
     Guatemala                         269.0                 70.0   
     Ecuador                            64.0                 66.0   
...                                      ...                  ...   
2010 Swaziland                           NaN                  NaN   
     Falkland Islands                    NaN                  NaN   
     Slovenia                            NaN                  NaN   
     Bahrain                             1.0                  NaN   
     French Guiana                       NaN                  NaN   

                       Pros. Discretion  
Year Nationality                         
2022 Venezuela                      NaN  
     Honduras                      13.0  
     Cuba                           1.0  
     Guatemala                     28.0  
     Ecuador                       28.0  
...                                 ...  
2010 Swaziland                      1.0  
     Falkland Islands               NaN  
     Slovenia                       NaN  
     Bahrain                        NaN  
     French Guiana                  NaN  

[4215 rows x 7 columns]

Make sure that every year has same country rows and every country has the same decision columns

In [2]:
# Rectify missing country rows
unique_index1 = data_df.index.unique(0)
unique_index2 = data_df.index.unique(1)

new_index = pd.MultiIndex.from_product([unique_index1, unique_index2])

data_df = data_df.reindex(new_index, axis='index')


with pd.option_context('display.max_rows', 250):
    display(data_df.loc[(2015)]['Removal Order'])

Nationality
Venezuela                                       53.0
Honduras                                      4053.0
Cuba                                            63.0
Guatemala                                     3019.0
Ecuador                                        424.0
El Salvador                                   4468.0
Brazil                                          43.0
Nicaragua                                      115.0
Mexico                                        4092.0
Haiti                                          235.0
India                                          284.0
Colombia                                        85.0
Russia                                          38.0
China                                          852.0
Peru                                            55.0
Turkey                                          15.0
Chile                                            3.0
Bangladesh                                     285.0
Uzebekistan                       

In [3]:
# Change all NaN values to 0
data_df = data_df.fillna(value=0.0)

# Re-order DF by year (ascending) and country (ascending)
data_df = data_df.sort_index()
with pd.option_context('display.max_rows', 250):
    display(data_df)


Pending  Grant Relief  Removal Order  Other Closure  \
Year Nationality                                                             
2001 Afghanistan           1.0         114.0           63.0           12.0   
     Albania               3.0         941.0          740.0           40.0   
     Algeria               0.0          24.0           27.0            2.0   
     Andorra               0.0           0.0            5.0            0.0   
     Angola                1.0           9.0           27.0            3.0   
...                        ...           ...            ...            ...   
2023 Withheld by EOIR      0.0           0.0            0.0            0.0   
     Yemen                 7.0           0.0            0.0            0.0   
     Zaire                 0.0           0.0            0.0            0.0   
     Zambia                0.0           0.0            0.0            0.0   
     Zimbabwe              6.0           0.0            0.0            0.0   

                       Terminate Proceedings  Voluntary Departure  \
Year Nationality                                                    
2001 Afghanistan                        11.0                  4.0   
     Albania                           127.0                 79.0   
     Algeria                             4.0                  3.0   
     Andorra                             1.0                  0.0   
     Angola                              2.0                  9.0   
...                                      ...                  ...   
2023 Withheld by EOIR                    0.0                  0.0   
     Yemen                               0.0                  0.0   
     Zaire                               0.0                  0.0   
     Zambia                              0.0                  0.0   
     Zimbabwe                            0.0                  0.0   

                       Pros. Discretion  
Year Nationality                         
2001 Afghanistan                    0.0  
     Albania                        1.0  
     Algeria                        0.0  
     Andorra                        0.0  
     Angola                         0.0  
...                                 ...  
2023 Withheld by EOIR               0.0  
     Yemen                          0.0  
     Zaire                          0.0  
     Zambia                         0.0  
     Zimbabwe                       0.0  

[5497 rows x 7 columns]

Merge Duplicate Country Names (identified manually)

In [4]:
# Remaining duplicates
dups = [
    ['Byelorussia (Belarus)', 'Belarus'],
    ['Holland', 'Netherlands'],
    ['Be Removed Fr...United States', 'Be Removed From the United States'],
    ['British India...ean Territory', 'British Indian Ocean Territory'],
    ['Federated Sta...of Micronesia', 'Federated States of Micronesia'],
    ['St. Vincent a...he Grenadines', 'St. Vincent and the Grenadines'],
    ['Stateless - A...ame A Country', 'Stateless - Alien Unable To Name A Country'],
    ['The Republic ...shall Islands', 'The Republic of the Marshall Islands']
]
for year in data_df.index.unique(0):
    for dup, orig in dups:
        if (dup in data_df.loc[year].index.unique(0)):
            data_df.rename(index={dup: orig}, inplace=True)   
            data_df = data_df.groupby(level=[0,1]).sum()                              # Sum the now-identical rows

Assign Regional Names to Countries (identified manually)

In [5]:
region_df = pd.read_csv('possible_nationalities.csv', usecols=[0,1,2])
region_df.set_index('Nationality', inplace=True)

data_df = data_df.join(region_df)

In [7]:
data_df.set_index('Region', append=True, inplace=True)
data_df.set_index('Subregion', append=True, inplace=True)
data_df = data_df.reorder_levels(['Year', 'Region', 'Subregion', 'Nationality'])
data_df.sort_index(inplace=True)

In [8]:
with pd.option_context('display.max_rows', 250):
    with pd.option_context('display.expand_frame_repr', True):
        with pd.option_context('display.width', 1000):
            display(data_df.loc[(2021)][['Removal Order', 'Grant Relief']])

Removal Order  Grant Relief
Region          Subregion              Nationality                                                            
Africa          Central Africa         Burundi                                               3.0           5.0
                                       Central African Republic                              0.0           0.0
                                       Congo                                                 6.0          11.0
                                       Democratic Republic of Congo                         23.0          14.0
                                       Rwanda                                                2.0           5.0
                                       South Sudan                                           5.0           2.0
                                       Zaire                                                 0.0           0.0
                Eastern Africa         Comoro Islands                                        0.0           0.0
                                       Djibouti                                              1.0           1.0
                                       Eritrea                                               5.0          61.0
                                       Ethiopia                                              7.0          37.0
                                       Kenya                                                 8.0           9.0
                                       Madagascar                                            0.0           0.0
                                       Malawi                                                0.0           0.0
                                       Mozambique                                            0.0           0.0
                                       Seychelles                                            0.0           0.0
                                       Somalia                                              14.0          14.0
                                       Tanzania                                              0.0           0.0
                                       Uganda                                                6.0          20.0
                Northern Africa        Algeria                                               0.0           0.0
                                       Chad                                                  0.0           1.0
                                       Egypt                                                17.0          38.0
                                       Libya                                                 2.0           2.0
                                       Mauritania                                            7.0           1.0
                                       Morocco                                               4.0           2.0
                                       Sudan                                                11.0          17.0
                                       Tunisia                                               1.0           0.0
                Southern Africa        Angola                                               10.0           0.0
                                       Botswana                                              0.0           0.0
                                       Lesotho                                               0.0           0.0
                                       Namibia                                               0.0           0.0
                                       South Africa                                          2.0           5.0
                                       Swaziland                                             0.0           0.0
                                       Zambia                                                1.0           0.0
                                       Zimbabwe                                        

Add a total column to make plotting easier

In [9]:
data_df['Total'] = data_df.sum(axis=1)

In [10]:
data_df

Pending  \
Year Region        Subregion             Nationality                                           
2001 Africa        Central Africa        Burundi                                         0.0   
                                         Central African Republic                        0.0   
                                         Congo                                           0.0   
                                         Democratic Republic of Congo                    1.0   
                                         Rwanda                                          0.0   
...                                                                                      ...   
2023 South America Western South America Peru                                         1063.0   
     Stateless     Stateless             No Nationality                                  0.0   
                                         Stateless - Alien Unable To Name A Country      9.0   
     Subantarctica Subantarctica         Bouvet Island                                   1.0   
                                         Heard and Mcdonald Islands                      0.0   

                                                                                     Grant Relief  \
Year Region        Subregion             Nationality                                                
2001 Africa        Central Africa        Burundi                                             21.0   
                                         Central African Republic                             4.0   
                                         Congo                                              149.0   
                                         Democratic Republic of Congo                         9.0   
                                         Rwanda                                              30.0   
...                                                                                           ...   
2023 South America Western South America Peru                                                 0.0   
     Stateless     Stateless             No Nationality                                       0.0   
                                         Stateless - Alien Unable To Name A Country           0.0   
     Subantarctica Subantarctica         Bouvet Island                                        0.0   
                                         Heard and Mcdonald Islands                           0.0   

                                                                                     Removal Order  \
Year Region        Subregion             Nationality                                                 
2001 Africa        Central Africa        Burundi                                              26.0   
                                         Central African Republic                              1.0   
                                         Congo                                               144.0   
                                         Democratic Republic of Congo                         19.0   
                                         Rwanda                                               22.0   
...                                                                                            ...   
2023 South America Western South America Peru                                                 21.0   
     Stateless     Stateless             No Nationality                                        0.0   
                                         Stateless - Alien Unable To Name A Country            0.0   
     Subantarctica Subantarctica         Bouvet Island                                         0.0   
                                         Heard and Mcdonald Islands                            0.0   

                                                                                     Other Closure  \
Year Region        Subregion             Nationality                                         

Convert all floats to int (cannot have fractions of people)

In [11]:
float_cols = data_df.select_dtypes(include=['float64']) # This will select float columns only
for col in float_cols.columns.values:
    data_df[col] = data_df[col].astype('int64')

data_df

Pending  \
Year Region        Subregion             Nationality                                           
2001 Africa        Central Africa        Burundi                                           0   
                                         Central African Republic                          0   
                                         Congo                                             0   
                                         Democratic Republic of Congo                      1   
                                         Rwanda                                            0   
...                                                                                      ...   
2023 South America Western South America Peru                                           1063   
     Stateless     Stateless             No Nationality                                    0   
                                         Stateless - Alien Unable To Name A Country        9   
     Subantarctica Subantarctica         Bouvet Island                                     1   
                                         Heard and Mcdonald Islands                        0   

                                                                                     Grant Relief  \
Year Region        Subregion             Nationality                                                
2001 Africa        Central Africa        Burundi                                               21   
                                         Central African Republic                               4   
                                         Congo                                                149   
                                         Democratic Republic of Congo                           9   
                                         Rwanda                                                30   
...                                                                                           ...   
2023 South America Western South America Peru                                                   0   
     Stateless     Stateless             No Nationality                                         0   
                                         Stateless - Alien Unable To Name A Country             0   
     Subantarctica Subantarctica         Bouvet Island                                          0   
                                         Heard and Mcdonald Islands                             0   

                                                                                     Removal Order  \
Year Region        Subregion             Nationality                                                 
2001 Africa        Central Africa        Burundi                                                26   
                                         Central African Republic                                1   
                                         Congo                                                 144   
                                         Democratic Republic of Congo                           19   
                                         Rwanda                                                 22   
...                                                                                            ...   
2023 South America Western South America Peru                                                   21   
     Stateless     Stateless             No Nationality                                          0   
                                         Stateless - Alien Unable To Name A Country              0   
     Subantarctica Subantarctica         Bouvet Island                                           0   
                                         Heard and Mcdonald Islands                              0   

                                                                                     Other Closure  \
Year Region        Subregion             Nationality                                         

Save pre-processed dataframe to disk

In [12]:
data_df.to_hdf('TRAC_AsylumFilings_Cleaned.hdf', key='AsylumFilings_2001to2023')